In [14]:
import joblib
import re
import string
import jieba
import pandas as pd
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm import tqdm
tqdm.pandas()
from transformers import PreTrainedTokenizerFast
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [15]:
# 请修改测试集文件名
test_dir = 'test.csv'

In [16]:
# 如测试集中不存在topic列，请使用 test = df[['content']]
df = pd.read_csv(test_dir)
test = df[['url','content', 'topic']]
#test = df[['content']]

In [17]:
non_chinese_content = test[~test['content'].str.contains(r'[\u4e00-\u9fff]')]
non_chinese_content

,url,content,topic
123,https://www.ximalaya.com/sound/611072245,China aims to increase the share of electric v...,2
2723,https://m.gmw.cn/2023-12/25/content_1303611500...,е…ЁйғЁеҜјиҲӘж—¶ж”ҝеӣҪеҶ…еӣҪйҷ…иҙўз»ҸзӨҫдјҡеЁұд...,18
2859,https://www.ximalaya.com/sound/592093182,China will launch the three astronauts of theS...,19


In [18]:
import requests

# DeepL API endpoint
DEEPL_API_URL = "https://api-free.deepl.com/v2/translate"

DEEPL_API_KEY = "67e30ce8-6c20-49a9-917b-fc252b83adff:fx"


def translate_text(text, target_lang="ZH"):
    """
    使用DeepL API翻译文本

    Args:
        text (str): 需要翻译的中文文本
        target_lang (str): 目标语言代码，默认为英语"EN"

    Returns:
        str: 翻译后的文本
    """
    params = {
        "auth_key": DEEPL_API_KEY,
        "text": text,
        "target_lang": target_lang,
        "source_lang": "EN"  # 指定源语言为中文
    }

    response = requests.post(DEEPL_API_URL, data=params)
    response_json = response.json()

    # 返回翻译结果
    return response_json["translations"][0]["text"] if response.status_code == 200 else None

In [19]:
non_chinese_content['content'] = non_chinese_content['content'].progress_apply(translate_text)
non_chinese_content

100%|██████████| 3/3 [00:08<00:00,  2.77s/it]
C:\Users\shens\AppData\Local\Temp\ipykernel_465440\2227448705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_chinese_content['content'] = non_chinese_content['content'].progress_apply(translate_text)


,url,content,topic
123,https://www.ximalaya.com/sound/611072245,在新能源汽车蓬勃发展的背景下，中国加快了推动绿色转型的步伐，其目标是到 2025 年，在公交...,2
2723,https://m.gmw.cn/2023-12/25/content_1303611500...,е…ЁйғЁеҜјиҲӘж—¶ж”ҝеӣҪеҶ…еӣҪйҷ…иҙўз»ҸзӨҫдјҡеЁұд...,18
2859,https://www.ximalaya.com/sound/592093182,据计划官员称，中国将于周二晚发射神舟十五号任务的三名宇航员前往空间站。中国载人航天工程办公室...,19


In [20]:
test.update(non_chinese_content)

In [21]:
def get_stopword():
    with open('baidu_stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f)
    stopwords.update(["年","月","日"])
    return stopwords

def wordopt_cn(text):
    # replace punctuation with space
    no_punct = re.sub(pattern, ' ', text)
    return str(no_punct)

def remove_stopwords(words):
    return (word for word in words if word not in stopwords)

def process_content(content):
    content = wordopt_cn(content)
    words = jieba.cut(content)
    words = remove_stopwords(words)
    content = ' '.join(words)
    content = re.sub(spaces_pattern, ' ', content)
    return content

# Precompile the regular expressions
punctuation = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." + string.punctuation
pattern = re.compile('[%s]' % re.escape(punctuation))
spaces_pattern = re.compile(r'\s+')
stopwords = get_stopword()

In [22]:
test['original_content'] = test['content']
test['content'] = test['content'].progress_apply(process_content)

C:\Users\shens\AppData\Local\Temp\ipykernel_465440\2199731547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['original_content'] = test['content']
100%|██████████| 3405/3405 [00:26<00:00, 130.53it/s]
C:\Users\shens\AppData\Local\Temp\ipykernel_465440\2199731547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['content'] = test['content'].progress_apply(process_content)


In [23]:
def dummy(text):
    return text
#test=pd.read_csv('test_jieba.csv')
vectorizer = joblib.load('vectorizer_v3.pkl')
model = joblib.load('ensemble_v3.pkl')

In [24]:
LOWERCASE = False
VOCAB_SIZE = 100_000
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['content']])


def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["content"]


raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['content'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))
    
tf_test = vectorizer.transform(tokenized_texts_test)

100%|██████████| 3405/3405 [00:04<00:00, 828.89it/s] 


In [26]:
import numpy as np
pred = model.predict(tf_test)
prob = model.predict_proba(tf_test)
test["pred"] = pred
# 概率最大的类别
test["prob_top1"] = prob.max(axis=1)
# 概率第二大的类别
test["pred_top2"] = prob.argsort(axis=1)[:, -2]
test["prob_top2"] = prob[np.arange(len(prob)), test["pred_top2"]]

In [29]:
test

,url,content,topic,original_content,pred,prob_top1,pred_top2,prob_top2
0,https://news.bjd.com.cn/2023/12/31/10663522.shtml,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,22,0.699783,1,0.267317
1,https://news.cctv.com/2023/11/17/ARTIMGJYjCtTJ...,当地 时间 11 16 习近平 主席 亚太经合组织 APEC 工商 领导人 峰会 发表 书面...,1,当地时间11月16日，习近平主席向亚太经合组织（APEC）工商领导人峰会发表书面演讲，强调我...,1,0.992075,0,0.001749
2,https://www.chinacourt.org/article/detail/2023...,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1,亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。国家主席习近平出席会议并发...,1,0.993566,0,0.002441
3,https://news.cctv.com/2023/11/17/ARTIvf5o43PK5...,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 2...,1,习近平出席亚太经合组织领导人同东道主嘉宾非正式对话会暨工作午宴央视网2023年11月17日 ...,1,0.984153,0,0.004991
4,http://politics.people.com.cn/n1/2023/1111/c10...,中国 国家 主席 习近平 11 14 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会...,1,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1,0.580717,4,0.406247
...,...,...,...,...,...,...,...,...
3400,https://zhuanlan.zhihu.com/p/667460523,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23,从头到尾来讲一遍，咱们按照惯例，先谈谈缅甸北部的那场风波。这场风波的源头是藏匿在缅甸北部的一...,23,0.886486,0,0.101815
3401,https://www.sohu.com/a/735213901_121451179,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23,缅北全线激战，敏昂莱誓言反击，冲突可能扩大化，中缅边境乱不得这几天，缅甸内战的爆发，引起了外...,23,0.993304,0,0.002566
3402,https://www.163.com/dy/article/IJ95N89T0553S2O...,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23,近日，缅甸内战再次卷土重来，引起了国际社会广泛的关注。然而，这次的冲突异常纷乱，各种消息传来...,23,0.993926,0,0.001408
3403,https://zhuanlan.zhihu.com/p/666103475,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23,缅北战火越烧越大，中方做两手准备，高层直接去了缅甸首都清酒半栩换种角度分析故事，带你了解更有...,23,0.981772,22,0.007741


In [30]:
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9756240822320118
Precision: 0.9758126998879221
Recall: 0.9756240822320118
F1: 0.975626535525693


In [32]:
low_prob = test[test['prob_top1'] < 0.5]
low_prob

,url,content,topic,original_content,pred,prob_top1,pred_top2,prob_top2
40,https://visionthai.net/zh-hans/article/srettha...,2023 亚太经合 会 APEC 领袖 峰会 今 15 日于 美国 旧金山 登场 泰国 总理...,1,2023年亚太经合会APEC领袖峰会今(15)日于美国旧金山登场，泰国总理赛塔(Settha...,0,0.473018,1,0.269182
82,https://new.qq.com/rain/a/20231110A08F9800,美国 已 中 俄 发出 邀请 普京 本人 去不去 俄罗斯 会 悬念 留 亚太经合组织 APE...,1,美国已向中俄发出邀请，普京本人去不去，俄罗斯会把悬念留到最后今年的亚太经合组织（APEC）峰...,3,0.363054,1,0.324839
148,https://www.thepaper.cn/newsDetail_forward_230...,微软 采购 核聚变 初创 公司 Helion 电力 Helion 预计 2028 年前 上线...,2,微软将采购由核聚变初创公司Helion产生的电力，Helion预计2028年前上线50兆瓦核...,0,0.395752,2,0.275005
252,https://new.qq.com/rain/a/20230107A07JAE00,2023 新德里 G20 领导人 峰会 成员国 名单 特邀 成员国 名单 印度 拟定 20 ...,3,2023年在新德里举行的G20领导人峰会成员国名单以及特邀成员国名单，印度已经拟定出来，除了...,3,0.461361,6,0.442152
261,https://www.sohu.com/a/748635250_121660501,印度 新德里 G20 峰会 落幕 国际 政坛 落幕 各国 领导人 展现 风采 机会 英国 头...,3,印度新德里G20峰会的落幕，不仅是国际政坛的落幕，也是各国领导人展现风采的机会。英国头号夫人...,3,0.434066,14,0.140028
293,https://news.sina.com.cn/w/2024-01-03/doc-inaa...,2024 1 1 正值 巴西 之子 卢拉 重返 总统 职位 一周年 纪念日 西班牙 《 国家...,3,2024年1月1日，正值“巴西之子”卢拉重返总统职位一周年的纪念日。西班牙《国家报》报道称，...,3,0.222040,11,0.112407
362,https://www.bbc.com/zhongwen/trad/world-63650348,G20 巴厘 島峰會 宣言 最 強烈 措辭 譴責 俄羅斯 解讀 時期 大國共識 陳 岩 BB...,3,G20巴厘島峰會宣言「以最強烈措辭」譴責俄羅斯，解讀特殊時期的大國共識陳岩BBC中文 發自印...,3,0.495803,0,0.159648
380,https://new.qq.com/rain/a/20231028A07CYG00,中方 同意 美方 共同努力 促成 两 国 元首 会晤 观察者 网 报道 日前 美国 总统 拜...,4,中方同意与美方共同努力，促成两国元首会晤据观察者网报道，日前美国总统拜登已经在白宫会见了访美...,3,0.458752,4,0.376438
401,https://www.zhihu.com/question/450134525,凤凰网 ​ 提问 美国 国际 关系 国际 政治 中美关系 ‭ 王毅 看待 中 美 首次 会谈...,4,凤凰网​的提问美国国际关系国际政治中美关系‭王毅如何看待中美首次会谈以及达成的结果，还有哪些...,4,0.232582,5,0.196238
483,https://iis.fudan.edu.cn/24/42/c38284a468034/p...,11 14 世界 目光 投向 印尼 巴厘岛 一场 重磅 峰会 登场 二 十国集团 G20 峰...,4,11月14日，世界目光投向印尼巴厘岛，一场重磅峰会在此登场。在二十国集团（G20）峰会前夕，...,4,0.498739,3,0.455673


In [33]:
from sklearn.metrics.pairwise import cosine_similarity
from text2vec import SentenceModel
from tqdm import tqdm
tqdm.pandas()
import numpy as np
m = SentenceModel("shibing624/text2vec-base-chinese-paraphrase", max_seq_length=512, device="cpu")
cls_emb_center = joblib.load("cls_emb_center_512.pkl")

test_emb = [m.encode(i) for i in tqdm(low_prob['original_content'])]

preds = []
max_sim = []
for i in test_emb:
    sim = [cosine_similarity(i.reshape(1, -1), cls_emb_center[j].reshape(1, -1))[0][0] for j in range(24)]
    max_sim.append(max(sim))
    pred = np.argmax(sim)
    preds.append(pred)

low_prob['pred'] = preds
low_prob["max_sim"] = max_sim

C:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /shibing624/text2vec-base-chinese-paraphrase/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025E625FD410>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 85684f45-b0b9-4cb8-93de-2b1bf23db861)')' thrown while requesting HEAD https://huggingface.co/shibing624/text2vec-base-chinese-paraphrase/resolve/main/tokenizer_config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /shibing624/text2vec-base-chinese-paraphrase/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.

In [34]:
low_prob_diff = low_prob[low_prob['topic'] != low_prob['pred']]
low_prob_diff

,url,content,topic,original_content,pred,prob_top1,pred_top2,prob_top2,max_sim
82,https://new.qq.com/rain/a/20231110A08F9800,美国 已 中 俄 发出 邀请 普京 本人 去不去 俄罗斯 会 悬念 留 亚太经合组织 APE...,1,美国已向中俄发出邀请，普京本人去不去，俄罗斯会把悬念留到最后今年的亚太经合组织（APEC）峰...,4,0.363054,1,0.324839,0.879174
148,https://www.thepaper.cn/newsDetail_forward_230...,微软 采购 核聚变 初创 公司 Helion 电力 Helion 预计 2028 年前 上线...,2,微软将采购由核聚变初创公司Helion产生的电力，Helion预计2028年前上线50兆瓦核...,0,0.395752,2,0.275005,0.817163
483,https://iis.fudan.edu.cn/24/42/c38284a468034/p...,11 14 世界 目光 投向 印尼 巴厘岛 一场 重磅 峰会 登场 二 十国集团 G20 峰...,4,11月14日，世界目光投向印尼巴厘岛，一场重磅峰会在此登场。在二十国集团（G20）峰会前夕，...,3,0.498739,3,0.455673,0.958168
1256,https://zhuanlan.zhihu.com/p/462566822,苏联 历次 对外 战争 理由 开战 方式 梳理 无 处天 战争 一个 国家 来说 生死攸关 ...,11,苏联历次对外战争的理由及开战方式梳理无所有处天战争对于一个国家来说，往往是生死攸关的大事，所...,6,0.306728,11,0.214002,0.883002
1285,http://fec.mofcom.gov.cn/article/fwydyl/zgzx/2...,叙利亚 驻华大使 一带 一路 使 重建 中 叙利亚 受益匪浅 2022 1 12 中国 叙利...,11,叙利亚驻华大使：加入“一带一路”将使重建中的叙利亚受益匪浅。2022年1月12日，中国和叙利...,22,0.467451,22,0.281272,0.888616
1450,https://www.dw.com/zh/%E5%8F%B0%E6%B9%BE%E5%A4...,台湾 国防部 12 24 日称 24 小时 中 歼 10 歼 11 歼 16 各型 共 8 ...,12,台湾国防部12月24日称，在过去24小时中，又有歼-10、歼-11、歼-16等各型共8架战机...,19,0.284248,12,0.257995,0.808087
1888,https://cn.chinadaily.com.cn/a/202401/03/WS659...,新 时代 中国 山水 披绿纱 引来 德企 大当家 来自 德国 柯雪婷 Kerstin Kae...,0,新时代，我在中国 | 山水披绿纱，引来德企大当家来自德国的柯雪婷（Kerstin Kaehl...,22,0.367663,8,0.092623,0.786593
1929,https://www.mem.gov.cn/xw/yjglbgzdt/202311/t20...,10 月份 我国 自然灾害 台风 洪涝灾害 风雹 低温 冷冻 雪灾 地震 地质灾害 森林 火...,0,10月份，我国自然灾害以台风、洪涝灾害为主，风雹、低温冷冻、雪灾、地震、地质灾害和森林火灾等...,18,0.489056,0,0.333686,0.899937
1967,http://jtt.hebei.gov.cn/jtyst/jtzx/styw/101680...,5 6 省厅 印发 《 做好 2023 年度 防汛防台 风 工作 通知 全力 做好 防汛防台...,0,5月6日，省厅印发《关于做好2023年度防汛防台风工作的通知》，就全力做好防汛防台风，确保人...,17,0.352868,17,0.293375,0.875500
2025,https://finance.sina.com.cn/wm/2023-07-28/doc-...,天津 启动 防洪 Ⅳ 级 应急 响应 明 持续 强降雨 下 台风 已 生成 天津市 防办 发...,0,天津启动防洪Ⅳ级应急响应！明起持续强降雨！下一台风已生成！天津市防办发布通知7月28日11时...,17,0.440685,0,0.402234,0.913465


In [35]:
test.update(low_prob[['pred']])
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.976798825256975
Precision: 0.976924927039556
Recall: 0.976798825256975
F1: 0.9767930393822226


In [36]:
test

,url,content,topic,original_content,pred,prob_top1,pred_top2,prob_top2
0,https://news.bjd.com.cn/2023/12/31/10663522.shtml,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,22,0.699783,1,0.267317
1,https://news.cctv.com/2023/11/17/ARTIMGJYjCtTJ...,当地 时间 11 16 习近平 主席 亚太经合组织 APEC 工商 领导人 峰会 发表 书面...,1,当地时间11月16日，习近平主席向亚太经合组织（APEC）工商领导人峰会发表书面演讲，强调我...,1,0.992075,0,0.001749
2,https://www.chinacourt.org/article/detail/2023...,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1,亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。国家主席习近平出席会议并发...,1,0.993566,0,0.002441
3,https://news.cctv.com/2023/11/17/ARTIvf5o43PK5...,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 2...,1,习近平出席亚太经合组织领导人同东道主嘉宾非正式对话会暨工作午宴央视网2023年11月17日 ...,1,0.984153,0,0.004991
4,http://politics.people.com.cn/n1/2023/1111/c10...,中国 国家 主席 习近平 11 14 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会...,1,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1,0.580717,4,0.406247
...,...,...,...,...,...,...,...,...
3400,https://zhuanlan.zhihu.com/p/667460523,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23,从头到尾来讲一遍，咱们按照惯例，先谈谈缅甸北部的那场风波。这场风波的源头是藏匿在缅甸北部的一...,23,0.886486,0,0.101815
3401,https://www.sohu.com/a/735213901_121451179,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23,缅北全线激战，敏昂莱誓言反击，冲突可能扩大化，中缅边境乱不得这几天，缅甸内战的爆发，引起了外...,23,0.993304,0,0.002566
3402,https://www.163.com/dy/article/IJ95N89T0553S2O...,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23,近日，缅甸内战再次卷土重来，引起了国际社会广泛的关注。然而，这次的冲突异常纷乱，各种消息传来...,23,0.993926,0,0.001408
3403,https://zhuanlan.zhihu.com/p/666103475,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23,缅北战火越烧越大，中方做两手准备，高层直接去了缅甸首都清酒半栩换种角度分析故事，带你了解更有...,23,0.981772,22,0.007741


In [37]:
diff = test[test['topic'] != test['pred']]
diff

,url,content,topic,original_content,pred,prob_top1,pred_top2,prob_top2
0,https://news.bjd.com.cn/2023/12/31/10663522.shtml,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,22,0.699783,1,0.267317
7,https://new.qq.com/rain/a/20231018A08MIN00,APEC 峰会 前 公使 提出 四点 核心 利益 中 美 会晤 看 美 表现 一带 一路 国...,1,APEC峰会前，我公使提出四点核心利益，中美是否会晤，得看美表现“一带一路”国际高峰论坛取得...,4,0.590462,1,0.284999
43,https://news.cctv.com/2023/11/19/ARTI7YpXS4mLy...,焦点访谈 把舵 中美关系 领航 亚太 合作 央视网 2023 11 19 22 01 加载 ...,1,焦点访谈：把舵中美关系 领航亚太合作央视网2023年11月19日 22:01正在加载央视网消...,4,0.837395,1,0.138323
60,https://news.sina.com.cn/zx/gj/2023-11-14/doc-...,亚太 国家 地区 领导人 13 抵达 美国 亚太经合组织 APEC 第三十次 领导人 非正式...,1,随着亚太一些国家和地区领导人从13日开始抵达美国，亚太经合组织（APEC）第三十次领导人非正...,4,0.762252,1,0.189153
82,https://new.qq.com/rain/a/20231110A08F9800,美国 已 中 俄 发出 邀请 普京 本人 去不去 俄罗斯 会 悬念 留 亚太经合组织 APE...,1,美国已向中俄发出邀请，普京本人去不去，俄罗斯会把悬念留到最后今年的亚太经合组织（APEC）峰...,4,0.363054,1,0.324839
...,...,...,...,...,...,...,...,...
3096,https://www.cmse.gov.cn/xwzx/202211/t20221129_...,2022 11 28 上午 9 时 神舟 十五号 载人 飞行 新闻 发布会 酒泉卫星发射中心...,21,2022年11月28日上午9时，神舟十五号载人飞行任务新闻发布会在酒泉卫星发射中心召开，中国...,19,0.925985,21,0.062508
3099,http://news.cyol.com/gb/articles/2024-01/03/co...,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,2023年12月31日晚，国家主席习近平发表二〇二四年新年贺词。自2013年岁末起，习近平主...,19,0.490701,21,0.147679
3127,http://www.china.com.cn/opinion2020/2023-10/28...,神舟 十七号 发射成功 续写 飞天 梦想 建设 航天 强国 北京 时间 10 26 11 时...,21,神舟十七号发射成功：续写飞天梦想，建设航天强国。北京时间10月26日11时13分59秒，搭载...,20,0.937831,19,0.042744
3132,https://www.163.com/dy/article/IA3TOMIN0532CE7...,神舟 十六号 出舱 多个 首次 神 十七 10 发射 航天员 杨利伟 7 国外 航天 执行 ...,21,神舟十六号出舱，有多个首次！神十七10月发射，有航天员杨利伟？7月，国外的航天任务执行可能最...,20,0.692039,21,0.299740
